In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import keras
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [2]:
df1=pd.read_csv('train.csv')
df2=pd.read_csv('test.csv')

In [48]:
df11=df1.dropna(how='any')

In [49]:
df=df11.append(df2)

In [50]:
df.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True
11,0008_03,Europa,False,B/1/P,55 Cancri e,45.0,False,39.0,7295.0,589.0,110.0,124.0,Wezena Flatic,True


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10883 entries, 0 to 4276
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   10883 non-null  object 
 1   HomePlanet    10796 non-null  object 
 2   CryoSleep     10790 non-null  object 
 3   Cabin         10783 non-null  object 
 4   Destination   10791 non-null  object 
 5   Age           10792 non-null  float64
 6   VIP           10790 non-null  object 
 7   RoomService   10801 non-null  float64
 8   FoodCourt     10777 non-null  float64
 9   ShoppingMall  10785 non-null  float64
 10  Spa           10782 non-null  float64
 11  VRDeck        10803 non-null  float64
 12  Name          10789 non-null  object 
 13  Transported   6606 non-null   object 
dtypes: float64(6), object(8)
memory usage: 1.2+ MB


In [52]:
df.corr()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
Age,1.000000,0.070913,0.127135,0.041206,0.118141,0.103842
RoomService,0.070913,1.000000,-0.019348,0.061917,0.010245,-0.028603
FoodCourt,0.127135,-0.019348,1.000000,0.001882,0.226161,0.239478
ShoppingMall,0.041206,0.061917,0.001882,1.000000,0.015032,0.011744
Spa,0.118141,0.010245,0.226161,0.015032,1.000000,0.146734
VRDeck,0.103842,-0.028603,0.239478,0.011744,0.146734,1.000000


In [53]:
df.isnull().sum()

PassengerId        0
HomePlanet        87
CryoSleep         93
Cabin            100
Destination       92
Age               91
VIP               93
RoomService       82
FoodCourt        106
ShoppingMall      98
Spa              101
VRDeck            80
Name              94
Transported     4277
dtype: int64

df3=df.dropna(subset=['PassengerId','HomePlanet','CryoSleep','Cabin','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Name'])

In [54]:
#df.astype({'22': 'object'}).dtypes
#data['22'] = data['22'].astype('float')
df.astype('object').dtypes

PassengerId     object
HomePlanet      object
CryoSleep       object
Cabin           object
Destination     object
Age             object
VIP             object
RoomService     object
FoodCourt       object
ShoppingMall    object
Spa             object
VRDeck          object
Name            object
Transported     object
dtype: object

In [55]:
df3=df.drop(df.columns[[0,7,8,9,10,11,12]], axis=1)

In [56]:
df3['Age'] = df3['Age'].astype('object')

In [57]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10883 entries, 0 to 4276
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   HomePlanet   10796 non-null  object
 1   CryoSleep    10790 non-null  object
 2   Cabin        10783 non-null  object
 3   Destination  10791 non-null  object
 4   Age          10792 non-null  object
 5   VIP          10790 non-null  object
 6   Transported  6606 non-null   object
dtypes: object(7)
memory usage: 680.2+ KB


In [58]:
df3=pd.get_dummies(df3,drop_first=True)

In [59]:
df_train=df3[:6606]
df_test=df3[6606:]

In [62]:
x,y=df_train.drop('Transported_True',axis=1),df_train[['Transported_True']]

In [63]:
model = Sequential()
model.add(Dense(81,activation = "relu"))
model.add(Dense(40,activation = "relu"))
model.add(Dense(27,activation = "relu"))
model.add(Dense(19,activation = "relu"))
model.add(Dense(5,activation = "relu"))
model.add(Dense(3,activation = "relu"))
model.add(Dense(1))

In [64]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

x=np.asarray(x).astype(np.int)
y=np.asarray(y).astype(np.int)

In [65]:
history=model.fit(x,y,epochs=80,batch_size=12,verbose=1)

Epoch 1/80
551/551 [==============================] - 3s 4ms/step - loss: 7.7685 - accuracy: 0.4964
Epoch 2/80
551/551 [==============================] - 2s 4ms/step - loss: 7.7685 - accuracy: 0.4964
Epoch 3/80
551/551 [==============================] - 2s 4ms/step - loss: 7.7685 - accuracy: 0.4964
Epoch 4/80
551/551 [==============================] - 2s 4ms/step - loss: 7.7685 - accuracy: 0.4964
Epoch 5/80
551/551 [==============================] - 2s 3ms/step - loss: 7.7685 - accuracy: 0.4964
Epoch 6/80
551/551 [==============================] - 2s 3ms/step - loss: 7.7685 - accuracy: 0.4964
Epoch 7/80
551/551 [==============================] - 2s 3ms/step - loss: 7.7685 - accuracy: 0.4964
Epoch 8/80
551/551 [==============================] - 2s 3ms/step - loss: 7.7685 - accuracy: 0.4964
Epoch 9/80
551/551 [==============================] - 2s 3ms/step - loss: 7.7685 - accuracy: 0.4964
Epoch 10/80
551/551 [==============================] - 2s 3ms/step - loss: 7.7685 - accuracy: 0.4964

In [66]:
scores=model.evaluate(x,y)

207/207 [==============================] - 1s 2ms/step - loss: 7.7685 - accuracy: 0.4964


In [67]:
print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))

accuracy: 49.64%


In [68]:
model=Sequential()
model.add(Dense(12,input_dim=8,input_shape=(x.shape[1],),kernel_initializer='uniform',activation='relu'))
model.add(Dense(8,kernel_initializer='uniform',activation='relu'))
model.add(Dense(1,kernel_initializer='uniform',activation='sigmoid'))

In [69]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [70]:
model.fit(x,y,epochs=150,batch_size=10,verbose=1)

Epoch 1/150
661/661 [==============================] - 1s 2ms/step - loss: 0.6217 - accuracy: 0.6839
Epoch 2/150
661/661 [==============================] - 1s 2ms/step - loss: 0.5331 - accuracy: 0.7319
Epoch 3/150
661/661 [==============================] - 1s 2ms/step - loss: 0.4624 - accuracy: 0.7611
Epoch 4/150
661/661 [==============================] - 1s 2ms/step - loss: 0.3306 - accuracy: 0.8562
Epoch 5/150
661/661 [==============================] - 1s 2ms/step - loss: 0.1860 - accuracy: 0.9463
Epoch 6/150
661/661 [==============================] - 1s 2ms/step - loss: 0.1262 - accuracy: 0.9550
Epoch 7/150
661/661 [==============================] - 1s 2ms/step - loss: 0.1031 - accuracy: 0.9581
Epoch 8/150
661/661 [==============================] - 1s 2ms/step - loss: 0.0920 - accuracy: 0.9614
Epoch 9/150
661/661 [==============================] - 1s 2ms/step - loss: 0.0850 - accuracy: 0.9635
Epoch 10/150
661/661 [==============================] - 1s 2ms/step - loss: 0.0791 - accura

In [71]:
scores=model.evaluate(x,y)

207/207 [==============================] - 1s 2ms/step - loss: 0.0048 - accuracy: 0.9986


In [72]:
print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))

accuracy: 99.86%


In [80]:
df_test.drop('Transported_True',axis=1,inplace=True)

In [81]:
y_pred=model.predict(df_test)

134/134 [==============================] - 0s 2ms/step


In [82]:
y_pred.dtype

dtype('float32')

In [84]:
y_pred=y_pred.round()
y_pred

array([[0.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [88]:
def write_prediction(prediction, name):
    PassengerId = np.array(df2['PassengerId']).astype(int)
    solution = pd.DataFrame(prediction, PassengerId, columns = ['Transported'])
    solution.to_csv(name, index_label = ['PassengerId'])

In [89]:
write_prediction(y_pred, "My_output.csv")

In [91]:
sub=pd.read_csv('My_output.csv')

In [92]:
sub

,PassengerId,Transported
0,1301,0.0
1,1801,1.0
2,1901,1.0
3,2101,0.0
4,2301,0.0
...,...,...
4272,926602,0.0
4273,926901,0.0
4274,927101,1.0
4275,927301,1.0


In [94]:
numpy_array = sub['Transported']
boolean_array = numpy_array == 1
print(boolean_array)

0       False
1        True
2        True
3       False
4       False
        ...  
4272    False
4273    False
4274     True
4275     True
4276     True
Name: Transported, Length: 4277, dtype: bool


In [100]:
sub.loc[sub.Transported == 1, 'Transported'] = True

In [101]:
sub.loc[sub.Transported == 0, 'Transported'] = False

In [102]:
sub

,PassengerId,Transported
0,1301,False
1,1801,True
2,1901,True
3,2101,False
4,2301,False
...,...,...
4272,926602,False
4273,926901,False
4274,927101,True
4275,927301,True


In [110]:
sub.to_csv('My_output3.csv',index=False)

In [107]:
sub2=pd.read_csv('My_output3.csv')

In [108]:
sub2

,PassengerId,Transported
0,1301,False
1,1801,True
2,1901,True
3,2101,False
4,2301,False
...,...,...
4272,926602,False
4273,926901,False
4274,927101,True
4275,927301,True


In [111]:
sub2[['PassengerId','Transported']]=sub2[['PassengerId','Transported']].astype(str)

In [112]:
sub2.to_csv('My_output4.csv',index=False)

In [114]:
sub2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  4277 non-null   int64 
 1   Transported  4277 non-null   object
dtypes: int64(1), object(1)
memory usage: 67.0+ KB


In [115]:
sample_submission=pd.read_csv('sample_submission.csv')

In [116]:
sample_submission['Transported']=sub2['Transported']
sample_submission.to_csv('submission.csv',index=False)